In [1]:
#Importing necessary Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#Loading Dataset
data = pd.read_csv("dataset2.csv")
data.head()
df = pd.DataFrame(data)

In [3]:
#Generating Matrix of Features
X = df.drop(axis=1, labels='label')
X

,N,P,K,temperature,humidity,ph,rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536
1,85,58,41,21.770462,80.319644,7.038096,226.655537
2,60,55,44,23.004459,82.320763,7.840207,263.964248
3,74,35,40,26.491096,80.158363,6.980401,242.864034
4,78,42,42,20.130175,81.604873,7.628473,262.717340
...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507
2196,99,15,27,27.417112,56.636362,6.086922,127.924610
2197,118,33,30,24.131797,67.225123,6.362608,173.322839
2198,117,32,34,26.272418,52.127394,6.758793,127.175293


In [4]:
#Generating Dependent Variable Vectors
Y = df.drop(axis=1, labels=['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall'])
Y

,label
0,rice
1,rice
2,rice
3,rice
4,rice
...,...
2195,coffee
2196,coffee
2197,coffee
2198,coffee


In [5]:
import json
onehot = pd.get_dummies(Y)
target_labels = onehot.columns
target = onehot.values
ref = {}
count = 0
for i in target_labels:
    ref[count] = {}
    ref[count] = i
    count+=1
with open('ann.json','w') as f:
    json.dump(ref,f)

In [6]:
df['label'].value_counts()

rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: label, dtype: int64

In [7]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,target,test_size=0.2,random_state=0)
X_train

,N,P,K,temperature,humidity,ph,rainfall
1567,27,120,200,21.452787,90.745319,6.110219,116.703658
2031,81,36,38,23.765547,87.983299,6.334838,150.316615
2073,90,59,35,24.251335,89.864541,7.098228,175.174211
1374,81,16,45,26.904357,86.254262,6.727468,59.759800
279,47,80,77,17.182484,16.428918,7.561108,72.850173
...,...,...,...,...,...,...,...
1033,102,71,48,28.654563,79.286937,5.695268,102.463378
1731,34,68,51,27.347349,94.177567,6.687088,40.351531
763,35,64,15,28.474423,63.536045,6.500145,69.527441
835,39,65,23,25.434598,69.126134,7.685959,41.026829


In [8]:
X_test.reset_index(inplace=True)

In [9]:
print(Y_train)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
print(Y_test)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [11]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


In [12]:
#Initialising ANN
from keras.layers import Dense, Flatten, Dropout
ann = tf.keras.Sequential()
ann.add(Dense(256, activation='relu'))
ann.add(Dropout(0.25))
ann.add(Dense(128, activation='relu'))
ann.add(Dropout(0.25))
ann.add(Dense(54, activation='relu'))
ann.add(Dropout(0.25))
ann.add(Dense(22, activation='softmax'))

In [13]:
ann.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [14]:
history = ann.fit(X_train,Y_train,batch_size = 20, epochs = 100, validation_split=0.2, callbacks=[es, mc])

Epoch 1/100
69/71 [============================>.] - ETA: 0s - loss: 5.9189 - accuracy: 0.0913
Epoch 1: val_accuracy improved from -inf to 0.17330, saving model to best_model.h5
71/71 [==============================] - 1s 8ms/step - loss: 5.8577 - accuracy: 0.0930 - val_loss: 2.8136 - val_accuracy: 0.1733
Epoch 2/100
47/71 [==================>...........] - ETA: 0s - loss: 2.9018 - accuracy: 0.1670
Epoch 2: val_accuracy improved from 0.17330 to 0.25852, saving model to best_model.h5
71/71 [==============================] - 0s 4ms/step - loss: 2.8306 - accuracy: 0.1662 - val_loss: 2.3554 - val_accuracy: 0.2585
Epoch 3/100
60/71 [========================>.....] - ETA: 0s - loss: 2.5319 - accuracy: 0.2517
Epoch 3: val_accuracy improved from 0.25852 to 0.50000, saving model to best_model.h5
71/71 [==============================] - 0s 3ms/step - loss: 2.5320 - accuracy: 0.2507 - val_loss: 1.8100 - val_accuracy: 0.5000
Epoch 4/100
68/71 [===========================>..] - ETA: 0s - loss: 2.07

In [15]:
X_test

,index,N,P,K,temperature,humidity,ph,rainfall
0,1320,105,14,50,26.214884,87.688398,6.419052,59.655908
1,1367,91,12,46,24.644585,85.499382,6.343943,48.312190
2,1291,14,121,203,9.724458,83.747656,6.158689,74.464111
3,264,52,73,79,17.257695,18.749440,7.840339,94.002872
4,728,32,66,17,34.946616,65.267740,7.162358,70.141514
...,...,...,...,...,...,...,...,...
435,446,32,70,20,20.893427,46.248565,6.208843,195.569787
436,1005,93,94,53,25.866324,84.423793,6.079179,114.535750
437,1919,117,56,15,25.992374,77.054355,7.368258,89.118821
438,708,27,64,21,32.842130,68.684015,7.543804,73.671662


In [16]:
y_pred = ann.predict(X_test.drop(columns=['index']))
y_pred

14/14 [==============================] - 0s 998us/step


array([[1.63122723e-14, 2.66495306e-15, 6.74308085e-28, ...,
        7.84016255e-11, 1.59687540e-21, 1.00000000e+00],
       [2.58962180e-14, 1.32964581e-14, 1.81122556e-26, ...,
        3.48965301e-10, 1.04331346e-20, 1.00000000e+00],
       [1.68537028e-07, 4.39611518e-21, 2.18426439e-25, ...,
        1.53681498e-19, 2.18843502e-17, 4.83202518e-19],
       ...,
       [6.04489123e-24, 7.40187330e-14, 5.04267131e-15, ...,
        7.59939046e-25, 9.95665130e-24, 9.14137947e-15],
       [3.73844636e-11, 1.02722358e-10, 9.22728837e-01, ...,
        1.83963389e-09, 1.24378308e-09, 1.67982867e-11],
       [1.00000000e+00, 4.68591779e-18, 5.34392976e-36, ...,
        7.75567644e-20, 1.23898580e-25, 3.33117523e-19]], dtype=float32)

In [17]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2048      
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 54)                6966      
                                                                 
 dropout_2 (Dropout)         (None, 54)                0         
                                                                 
 dense_3 (Dense)             (None, 22)                1

In [18]:
from keras.utils.np_utils import to_categorical


a = [27.0, 120.0, 200.0, 21.4527, 90.74, 6.11, 116.703]
ann.predict([a])

1/1 [==============================] - 0s 91ms/step


array([[1.0000000e+00, 8.5434941e-19, 1.1166501e-36, 4.7542303e-14,
        2.8749479e-29, 2.1630571e-30, 1.0001375e-37, 5.0427587e-13,
        1.5154656e-21, 0.0000000e+00, 6.5885576e-29, 0.0000000e+00,
        4.4191145e-22, 1.9312492e-26, 8.5096757e-32, 2.8310361e-28,
        1.9253859e-34, 2.7552769e-14, 2.6122379e-33, 1.1195516e-19,
        2.0817200e-26, 1.9395471e-19]], dtype=float32)

In [19]:
prediction = ann.predict(X_test.drop(columns=['index']))
preds_idx = np.argsort(prediction)[:,:-20-1:-1]

14/14 [==============================] - 0s 1ms/step


In [20]:
with open ('ann.json','r') as p:
        brand_dict = json.load(p)

In [21]:
res = pd.DataFrame(ann.predict(X_test.drop(columns=['index'])),columns= list(brand_dict.values()))
import operator
print(res.columns)
dict_ = {}
res['score'] = 0
for index, row in res.iterrows():
    list_ = []
    dict_ = dict(row)
    sorted_dict = sorted(dict_.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = sorted_dict[0:20]
    for i in sorted_dict:
        if i[1]>=0.05:
#             print(i)
            list_.append({i[0]:i[1]})
#             print(list_)
            res['score'][index] = list_

14/14 [==============================] - 0s 1ms/step
Index(['label_apple', 'label_banana', 'label_blackgram', 'label_chickpea',
       'label_coconut', 'label_coffee', 'label_cotton', 'label_grapes',
       'label_jute', 'label_kidneybeans', 'label_lentil', 'label_maize',
       'label_mango', 'label_mothbeans', 'label_mungbean', 'label_muskmelon',
       'label_orange', 'label_papaya', 'label_pigeonpeas', 'label_pomegranate',
       'label_rice', 'label_watermelon'],
      dtype='object')


C:\Users\DELL\AppData\Local\Temp\ipykernel_3488\1757233222.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['score'][index] = list_


In [22]:
X_test['predicted_values'] = 0
for i in range(len(res['score'])):
    X_test['predicted_values'][i] = res['score'][i]

C:\Users\DELL\AppData\Local\Temp\ipykernel_3488\3865636814.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['predicted_values'][i] = res['score'][i]


In [29]:
test = pd.DataFrame(Y_test,columns= list(brand_dict.values()))
import operator
print(test.columns)
dict_ = {}
test['score'] = 0
for index, row in test.iterrows():
    list_ = []
    dict_ = dict(row)
    sorted_dict = sorted(dict_.items(), key=operator.itemgetter(1),reverse=True)
    sorted_dict = sorted_dict[0:20]
    for i in sorted_dict:
        if i[1]>=0.05:
#             print(i)
            list_.append({i[0]:i[1]})
#             print(list_)
            test['score'][index] = list_

test['score']

Index(['label_apple', 'label_banana', 'label_blackgram', 'label_chickpea',
       'label_coconut', 'label_coffee', 'label_cotton', 'label_grapes',
       'label_jute', 'label_kidneybeans', 'label_lentil', 'label_maize',
       'label_mango', 'label_mothbeans', 'label_mungbean', 'label_muskmelon',
       'label_orange', 'label_papaya', 'label_pigeonpeas', 'label_pomegranate',
       'label_rice', 'label_watermelon'],
      dtype='object')


C:\Users\DELL\AppData\Local\Temp\ipykernel_3488\915796639.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['score'][index] = list_


0      [{'label_watermelon': 1}]
1      [{'label_watermelon': 1}]
2          [{'label_grapes': 1}]
3        [{'label_chickpea': 1}]
4       [{'label_blackgram': 1}]
                 ...            
435    [{'label_pigeonpeas': 1}]
436        [{'label_banana': 1}]
437        [{'label_cotton': 1}]
438     [{'label_blackgram': 1}]
439         [{'label_apple': 1}]
Name: score, Length: 440, dtype: object

In [30]:
print(test)

     label_apple  label_banana  label_blackgram  label_chickpea  \
0              0             0                0               0   
1              0             0                0               0   
2              0             0                0               0   
3              0             0                0               1   
4              0             0                1               0   
..           ...           ...              ...             ...   
435            0             0                0               0   
436            0             1                0               0   
437            0             0                0               0   
438            0             0                1               0   
439            1             0                0               0   

     label_coconut  label_coffee  label_cotton  label_grapes  label_jute  \
0                0             0             0             0           0   
1                0             0           

In [31]:
X_test['actual_values'] = test['score']

In [32]:
X_test.to_csv('ann.csv')

In [33]:
X_test

,index,N,P,K,temperature,humidity,ph,rainfall,predicted_values,actual_values
0,1320,105,14,50,26.214884,87.688398,6.419052,59.655908,[{'label_watermelon': 1.0}],[{'label_watermelon': 1}]
1,1367,91,12,46,24.644585,85.499382,6.343943,48.312190,[{'label_watermelon': 1.0}],[{'label_watermelon': 1}]
2,1291,14,121,203,9.724458,83.747656,6.158689,74.464111,[{'label_grapes': 0.9999998807907104}],[{'label_grapes': 1}]
3,264,52,73,79,17.257695,18.749440,7.840339,94.002872,[{'label_chickpea': 1.0}],[{'label_chickpea': 1}]
4,728,32,66,17,34.946616,65.267740,7.162358,70.141514,[{'label_blackgram': 0.9552872180938721}],[{'label_blackgram': 1}]
...,...,...,...,...,...,...,...,...,...,...
435,446,32,70,20,20.893427,46.248565,6.208843,195.569787,[{'label_pigeonpeas': 0.9973413348197937}],[{'label_pigeonpeas': 1}]
436,1005,93,94,53,25.866324,84.423793,6.079179,114.535750,[{'label_banana': 0.9999864101409912}],[{'label_banana': 1}]
437,1919,117,56,15,25.992374,77.054355,7.368258,89.118821,"[{'label_cotton': 0.9191798567771912}, {'label...",[{'label_cotton': 1}]
438,708,27,64,21,32.842130,68.684015,7.543804,73.671662,"[{'label_blackgram': 0.9227288365364075}, {'la...",[{'label_blackgram': 1}]


In [34]:
model_json = ann.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
print("Saved model to disk")

Saved model to disk


In [35]:
values = [30, 50, 40, 30, 80, 6.5, 120]


result = np.array(ann.predict([values]))
print(result)

1/1 [==============================] - 0s 32ms/step
[[1.2417167e-05 1.3001543e-05 8.2960114e-06 4.5243934e-07 3.5751640e-04
  2.8172083e-06 1.4734485e-06 2.0972912e-07 1.1331852e-02 2.7051709e-07
  1.0351517e-07 5.5715729e-05 8.8639284e-04 3.0840478e-05 5.7481867e-07
  4.3539725e-07 5.7111424e-06 9.8540568e-01 2.7332988e-04 1.3510077e-03
  2.6000434e-04 1.7498149e-06]]
